In [5]:
import pandas as pd
import csv
import torch

In [6]:
decisions = pd.read_csv('decisions.csv')

Target decisions which went to hearing e.g. were not dismissed or settled

Reason Closed Key:	1‑Decision/Hearing | 2‑Decision/Evidence | 3‑Stipulation | 4‑Dismissal | 5‑Withdrawal | 6‑Duplicate | 7‑Premature | 8‑Rollover | 9‑Incomplete | A‑Void | B‑Dismissal/Lack Standing | C‑Dismissal/FTA

In [9]:
len(decisions)

122509

In [8]:
decisions.groupby('reason_code').size()

reason_code
1       3282
13A        3
14         1
15         1
1A        11
2      42630
24         7
24A        3
25         7
26         4
29         1
2A        73
2B         3
3      73926
34        11
345        1
34A        1
35        35
36         3
39         1
3A       147
3B         1
4          8
5          1
A          3
B          5
dtype: int64

In [11]:
valid_codes = ["1", "1A", "2", "2A"]
good_decisions = decisions[decisions["reason_code"].isin(valid_codes)]

In [12]:
len(good_decisions)

45996

In [198]:
file_loc = "files"
sections = ["findings of fact", "conclusion of law", "this is a final administrative"]
def get_decision(docket_name, debug=False):
    with open('files/' + docket_name + '.txt') as f:
        rslt = f.read()    
    rslt = rslt.replace('\n', '').lower().strip()
    
    if sections[0] in rslt:
        decision, s1 = rslt.split(sections[0])
    else:
        decision, s1 = rslt.split("analysis", 1)
    if debug:
        print(len(decision))
        print(len(s1))
        print(decision)
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print(s1)    
    if sections[1] in s1:
        fact, s2 = s1.split(sections[1])
    else:
        fact, s2 = s1.split("after reviewing the record")
    conclusion, s3 = s2.split(sections[2])
    
    return extract_outcome(decision), fact, conclusion

def extract_outcome(decision):
    if "no change" in decision:
        return "no change"
    elif "reduction" in decision:
        return "reduction"
    else:
        raise

In [199]:
bad_ls = []
output = []

for i, docket in enumerate(good_decisions["docket_name"]):
    try:
        decision = get_decision(docket)
        output.append((docket, decision[0], decision[1], decision[2]))
    except:
        bad_ls.append(docket)
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [206]:
len(bad_ls)

394

In [221]:
tmp = pd.DataFrame.from_records(output, columns = ['docket','outcome','facts','conclusion'])

binary_dict = {"reduction":1,
               "no change":0}

tmp['target'] = tmp.outcome.map(binary_dict)

tmp.to_csv('standardized.csv', index=False)
